# This example demonstrate how to train region classification (standart image classification model).

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
import os
import sys
import warnings
from matplotlib import pyplot as plt
warnings.filterwarnings('ignore')

# Nomeroff-Net path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [3]:
plt.rcParams["figure.figsize"] = (10, 10)

In [4]:
%matplotlib inline 

In [5]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# auto download latest dataset
info = modelhub.download_dataset_for_model("numberplate_orientations")
PATH_TO_DATASET = info["dataset_path"]
PATH_TO_TMP_CUSTOM_DATASET = f'{PATH_TO_DATASET}_custom'

# local path dataset
#PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/OptionsDetector/numberplate_options_example")

In [6]:
PATH_TO_DATASET

'/var/www/nomeroff-net24/nomeroff_net/tools/../../data/./dataset/InverseDetector/numberplate_orientations/autoriaNumberplateOrientationDataset-2021-09-12'

In [7]:
PATH_TO_TMP_CUSTOM_DATASET

'/var/www/nomeroff-net24/nomeroff_net/tools/../../data/./dataset/InverseDetector/numberplate_orientations/autoriaNumberplateOrientationDataset-2021-09-12_custom'

In [8]:
DATASET_NAME = "inverse"
VERSION = "2021_09_12_pytorch_lightning"

RESULT_PATH = os.path.join(NOMEROFF_NET_DIR, "./data/models/", 'numberplate_{}_{}.ckpt'.format(DATASET_NAME, VERSION))

In [9]:
from nomeroff_net import InverseDetector

orientations = [
    "0", 
    "180"
]


In [10]:
# definde your parameters
class MyInverseClassificator(InverseDetector):
    def __init__(self):
        InverseDetector.__init__(self)
        
        # outputs
        self.orientations = orientations
        
        self.epochs           = 100
        self.batch_size       = 64
        
        # count gpu for train
        self.gpus = 1
        
        self.height         = 64
        self.width         = 295


In [11]:
# initialize region detector.
inverseClassificator = MyInverseClassificator()
inverseClassificator.prepare(PATH_TO_DATASET, verbose=1, num_workers=10)

START PREPARING
DATA PREPARED


In [12]:
# train
lr_finder = inverseClassificator.tune()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type        | Params
-------------------------------------------
0 | inp_conv   | Conv2d      | 896   
1 | conv1      | Conv2d      | 18.5 K
2 | conv2      | Conv2d      | 73.9 K
3 | conv3      | Conv2d      | 147 K 
4 | pool       | MaxPool2d   | 0     
5 | dropout    | Dropout     | 0     
6 | fc1        | Linear      | 2.1 M 
7 | fc2        | Linear      | 131 K 
8 | batch_norm | BatchNorm1d | 1.0 K 
9 | fc3        | Linear      | 514   
-------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.885     Total estimated model params size (MB)


Restored states from the checkpoint file at /var/www/nomeroff-net24/train/lr_find_temp_model.ckpt
Learning rate set to 0.006918309709189364


In [13]:
# Results can be found in
lr_finder["lr_find"].results

# Plot with
fig = lr_finder["lr_find"].plot(suggest=True)
fig.show()

In [15]:
# train
model = inverseClassificator.train()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type        | Params
-------------------------------------------
0 | inp_conv   | Conv2d      | 896   
1 | conv1      | Conv2d      | 18.5 K
2 | conv2      | Conv2d      | 73.9 K
3 | conv3      | Conv2d      | 147 K 
4 | pool       | MaxPool2d   | 0     
5 | dropout    | Dropout     | 0     
6 | fc1        | Linear      | 2.1 M 
7 | fc2        | Linear      | 131 K 
8 | batch_norm | BatchNorm1d | 1.0 K 
9 | fc3        | Linear      | 514   
-------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.885     Total estimated model params size (MB)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



[INFO] best model path /var/www/nomeroff-net24/train/lightning_logs/version_116/checkpoints/epoch=78-step=54430.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9967619776725769, 'test_loss': 0.31560415029525757}
--------------------------------------------------------------------------------


In [ ]:
#inverseClassificator.prepare(PATH_TO_DATASET, verbose=1)

In [16]:
inverseClassificator.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9967619776725769, 'test_loss': 0.31560415029525757}
--------------------------------------------------------------------------------


[{'test_loss': 0.31560415029525757, 'test_accuracy': 0.9967619776725769}]

In [17]:
inverseClassificator.save(RESULT_PATH)

model save to /var/www/nomeroff-net24/./data/models/numberplate_inverse_2021_09_12_pytorch_lightning.ckpt
